# Objetivo:


Aqui realizamos o fine-tuning de um modelo LLM usando técnicas eficientes em termos de memória, como o LoRA (Low-Rank Adaptation).

O objetivo é ajustar um modelo existente para que ele seja capaz de gerar respostas específicas baseadas nos dados de produtos que iremos passar!

In [1]:
#Conexão com o Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Bibliotecas Utilizadas
**unsloth:**

Biblioteca utilizada para facilitar a implementação de modelos de linguagem e suporte a treinos com precisão reduzida (4 bits).


**transformers:**

Biblioteca usada para modelos de linguagem e treinamento.


**trl:**

Usada para treinamento supervisionado fino (SFT) de modelos de linguagem de maneira eficiente.


**datasets:**

Utilizada para carregar e manipular datasets, que também suporta a formatação dos dados para serem usados com modelos de linguagem.


**peft:**

Usada para aplicar técnicas de fine-tuning eficientes como LoRA.

In [2]:
# Instalação das bibliotecas necessárias
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets
!pip install triton


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-reus7lod/unsloth_30bd6c304d4644e484d952396bb9d146
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-reus7lod/unsloth_30bd6c304d4644e484d952396bb9d146
  Resolved https://github.com/unslothai/unsloth.git to commit d91d40a7b6b556f2d1fdd3e1e430f7a76a799627
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done



> Vamos carregar os modelos que suportam o fine-tuning e configurar nossos PATHs!



In [3]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# Caminho para o dataset processado no notebook anterior!
DATA_PATH = "/content/drive/MyDrive/Pós Tech - IA PARA DEVS/Pós Tech/FASE3/new_trn.json"

# Caminho para salvar nosso dataset final depois que fizermos a ultima transformacão!
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Pós Tech - IA PARA DEVS/Pós Tech/FASE3/formatted_trn.json"

# Comprimento máximo das sequências tokens de entrada para o modelo. 2048 é comum para grandes modelos de linguagem.
max_seq_length = 2048
# Tipo de dados usado para operações numéricas no modelo. Definido como 'None' aqui.
dtype = None
# Define que o modelo será carregado no modo de 4 bits, uma técnica para reduzir o uso de memória ao trabalhar com grandes modelos.
load_in_4bit = True

# Modelos que suportam o fine-tuning em 4 bits para uso eficiente de memória
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.



> Vamos agora formatar nosso dataset para seguir o formado de (Instructions, Inputs e Outputs)



In [4]:
# Função para formatar o dataset em um formato compatível com o modelo (Instructions, Inputs e Outputs)
def format_dataset_into_model_input(data):
    def separate_text(full_text):
        # Encontra as seções específicas de descrição e título do produto baseado nas "tags" que adicionamos na preparacão do dataset!
        descricoes_start = full_text.find("[|Descricão|]") + len("[|Descricão|]")
        descricoes_end = full_text.find("[|eDescricão|]")

        # Encontra as seções específicas de título do produto baseado nas "tags" que adicionamos na preparacão do dataset!
        titulo_start = full_text.find("[|Titulo|]") + len("[|Titulo|]")
        titulo_end = full_text.find("[|eTitulo|]")

        # Separa a instrução, descrição (input) e o título (output)
        instruction = full_text.split('\n')[0] # A primeira linha do texto é considerada a instrução
        input_text = full_text[descricoes_start:descricoes_end].strip() # A descrição do produto é extraída entre as tags de "Descricão"
        response = full_text[titulo_start:titulo_end].strip() # O título do produto é extraído entre as tags de "Titulo".

        #Retorna os valores que pegou.
        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando cada item do dataset de produtos
    for item in data:
        # Coleta o campo 'input' do item (o texto que contém as instruções e descrições formatadas).
        prompt = item['input']

        # Chama a função separate_text para dividir o prompt em instrução, entrada e resposta.
        instruction, input_text, response = separate_text(prompt)

        # Adiciona os resultados nas respectivas listas.
        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(response)

    # Criando o dicionário final com os dados formatados
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o dataset em um arquivo JSON no PATH que definimos nas primeiras etapas do código!
    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")



> Vamos carregar nosso modelo escolhido e chamar a funcão para formatar nosso dataset para o template de fine-tuning!



In [5]:
# Vamos agora chamar nosso método acima passando o dataset preparado anteriormente e formatando os dados!
with open(DATA_PATH, 'r', encoding='utf-8') as file:
    data = json.load(file) #Converte STR para JSON

    # Ajusta as informações para o formato esperado pelo modelo (instruções, entradas e saídas).
    format_dataset_into_model_input(data)

# Carregando o modelo e o tokenizador com suporte a 4 bits
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",  # Nome do modelo pré-treinado que será carregado.
    max_seq_length = max_seq_length,  # Comprimento máximo de tokens que o modelo pode processar.
    dtype = dtype,  # Tipo de dado numérico a ser utilizado.
    load_in_4bit = load_in_4bit,  # Define se o modelo será carregado no modo 4 bits para otimização de memória.
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!




> Configurando nosso modelo !



In [6]:
# Configurando o modelo com LoRA para fine-tuning
model = FastLanguageModel.get_peft_model(
    model,  # O modelo pré-treinado que será ajustado.

    # Parâmetro 'r' define o "rank" de LoRA, um valor mais alto aumenta a capacidade de aprendizado.
    r = 16,

    # Lista de módulos alvo que receberão as adaptações LoRA.
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",  # Módulos de atenção (query, key, value, output projections).
        "gate_proj", "up_proj", "down_proj"
    ],

    # Um valor mais alto aumenta a influência das adaptações durante o treinamento.
    lora_alpha = 16,

    # Dropout de 0 significa que não há regularização através de dropout.
    lora_dropout = 0,

    # Definição do uso de bias (viés) nas camadas LoRA. Aqui, "none" significa que não será usado bias adicional.
    bias = "none",

    # Ativa o "gradient checkpointing" para reduzir o uso de memória durante o treinamento.
    use_gradient_checkpointing = "unsloth",

    # Define random state aleatório para garantir a reprodutibilidade dos resultados do treinamento.
    random_state = 3407,

    # Especifica se o modelo usará rslora (uma variante do LoRA que não está ativa neste caso).
    use_rslora = False,

    # Configuração para quantização LoFTQ (Low-bit Floating Point Quantization), definido como None.
    loftq_config = None,
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
# Template de prompt para gerar as instruções, entradas e respostas!
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

# Função para formatar os prompts de treinamento
def formatting_prompts_func(examples):
    #Separa os valores em 3 variáveis
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []

    #Faz um loop entre todas as 3 variáveis, adicionando na lista de texts.
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

# Carregando o dataset formatado no path OUTPUT_PATH_DATASET.
dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
# Executa a funcão formatting_prompts_func para cada elemento do dataset!
dataset = dataset.map(formatting_prompts_func, batched = True,)

# Salvar o dataset final no drive!
output_filename = r'/content/drive/MyDrive/Pós Tech - IA PARA DEVS/Pós Tech/FASE3/dataset-treinamento-final.json'
with open(output_filename, 'w', encoding='utf-8') as file:
    json.dump(dataset, file, ensure_ascii=False, indent=4)

In [8]:
# Configurando o treinamento do modelo com o Trainer
trainer = SFTTrainer(
    model = model, #Nosso modelo previamente cofigurado.
    tokenizer = tokenizer, #Nosso tokenizador previamente configurado juntamente com o modelo.
    train_dataset = dataset, #Nosso dataset formatado carregado acima em OUTPUT_PATH_DATASET
    dataset_text_field = "text", #Nome do campo no dataset que contém o texto a ser processado.
    max_seq_length = max_seq_length, #Comprimento máximo de tokens que o modelo pode processar
    dataset_num_proc = 2, #Número de processos paralelos que pode ser usado pra processar o dataset.
    packing = False, #Define se vai tratar multiplas sequências menores em uma única maior como False.
    args = TrainingArguments(
        per_device_train_batch_size = 2, #Tamanho do batch de treinamento por dispositivo
        gradient_accumulation_steps = 4, #Número de passos que podem acumular antes do cálculo do gradiente.
        warmup_steps = 5, #Número de passos para aquecimento
        max_steps = 60, #Número máximo de passos
        learning_rate = 2e-4, #Taxa de aprendizado, um valor mais alto pode acelerar o treinamento, porém pode causar instabilidade.
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1, #Define a quantidade de passos para registrar infos de logs!
        optim = "adamw_8bit", #Otimizador usado no treinamento
        weight_decay = 0.01, #Fator de decaimento para evitar overfitting
        lr_scheduler_type = "linear",
        seed = 3407, #Random Seed para evitar aleatoriadade.
        output_dir = "outputs", #Diretório de salvamento dos checkpoints.
    ),
)

Map (num_proc=2):   0%|          | 0/2248619 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs




> Vamos finalmente treinar nosso modelo !!



In [9]:
# Iniciando o processo de treinamento
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,248,619 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.242500
2,3.081500
3,3.245200
4,2.727700
5,2.645500
6,2.496700
7,2.278400
8,1.838400
9,1.800000
10,1.807300




> Vamos testar agora enviando uma descricão para ver se o modelo nos responde com o título do produto correto !



In [13]:
# Preparando o modelo para inferência
FastLanguageModel.for_inference(model)

# Exemplo de inferência usando o modelo treinado
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Encontre o seguinte produto.",
        "In a distant, timeless place, a mysterious prophet walks the sands.", #output deve ser o título de produto "The Prophet"
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Encontre o seguinte produto.

### Input:
In a distant, timeless place, a mysterious prophet walks the sands.

### Response:
The Prophet<|end_of_text|>


> Legal ! Conseguimos chegar ao resultado que queríamos com esse input, o resultado foi o título de produto "The Prophet"





> Vamos agora salvar nosso modelo e nosso tokenizador!



In [14]:
# Salvando o modelo e o tokenizador treinados
model.save_pretrained("/content/drive/MyDrive/Pós Tech - IA PARA DEVS/Pós Tech/FASE3/lora_model") # Salvar o modelo
tokenizer.save_pretrained("/content/drive/MyDrive/Pós Tech - IA PARA DEVS/Pós Tech/FASE3/lora_model") #Salvar o tokenizador

('/content/drive/MyDrive/Pós Tech - IA PARA DEVS/Pós Tech/FASE3/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Pós Tech - IA PARA DEVS/Pós Tech/FASE3/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Pós Tech - IA PARA DEVS/Pós Tech/FASE3/lora_model/tokenizer.json')



> Vamos fazer a mesma coisa agora, só que dessa vez carregando o modelo e tokenizador que salvamos para pularmos toda a etapa de treinamento!



In [15]:
# Configurando nosso modelo diretamente pelo arquivo de lora_model que salvamos anteriormente
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/Pós Tech - IA PARA DEVS/Pós Tech/FASE3/lora_model", # Chamamos o modelo treinado que salvamos
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [16]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Encontre o seguinte produto.",
        "In a distant, timeless place, a mysterious prophet walks the sands.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Encontre o seguinte produto.

### Input:
In a distant, timeless place, a mysterious prophet walks the sands.

### Response:
The Prophet<|end_of_text|>


> Legal ! Obtivemos novamente o resultado que queríamos com esse input, o resultado foi o título de produto "The Prophet"